<a href="https://colab.research.google.com/github/deanodeano11/my-work-on-StyleGAN2-ada/blob/main/My_copy_of_woctezumas_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2-ADA: training a model from scratch

References:
- this notebook: https://github.com/woctezuma/steam-stylegan2-ada
- the original StyleGAN2-ADA repository: https://github.com/NVlabs/stylegan2-ada
- my fork of StyleGAN2-ADA: https://github.com/deanodeano11/stylegan2-ada.git
- my copy of train.py: https://github.com/deanodeano11/stylegan2ada-training-on-a-custom-network.git

I will annotate each code cell with the text cell above it


##Run these cells any time you want to use notebook


The Cells below: 
- Import google drive 
- Select the correct version of tensorflow
- Install Stylegan

Therefore they are necessary every time you start the virtual machine

### Mount Google Drive

Imports the Google Colab transfer module which we will use in the next few cells

In [ ]:
%pip install Google-Colab-Transfer

Makes use of the Google Colab transfer module to:
- set the home path of your google drive
- set the home path of the virtual machine.
- mount your google drive to the virtual machine (follow instructions)

In [ ]:
import colab_transfer

gd = colab_transfer.get_path_to_home_of_google_drive()
lm = colab_transfer.get_path_to_home_of_local_machine()

colab_transfer.mount_google_drive()

Mounted at /content/drive/


### Switch to Tensorflow 1.x

> TensorFlow 2.x is not supported.

Reference: https://github.com/NVlabs/stylegan2-ada#requirements

> Colab uses TensorFlow 2.x by default, though you can switch to 1.x by the method shown below.

Reference: https://colab.research.google.com/notebooks/tensorflow_version.ipynb

So simply run the code below to use 1.x

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


### Install my fork of StyleGAN2-ADA
Installing my StyleGan2-ada fork to the local machine

A syntax error kept occuring and so during this step i also replace train.py with my own version.

Therefore the instructions here are:
- clone my fork of stylegan2-ada
- clone my fixed train.py file
- replace the broken train.py file in stylegan2-ada with my fixed version

For reference:

%cd is for setting the current directory

%rm is for removing references

!git is for using git commands

!mv is for moving a file

!rm is for removing a file

In [ ]:
%cd /content/

%rm -rf stylegan2-ada/
!git clone https://github.com/deanodeano11/stylegan2-ada.git

%cd stylegan2-ada/
!git checkout google-colab
!rm /content/stylegan2-ada/train.py
!git clone https://github.com/deanodeano11/stylegan2ada-training-on-a-custom-network.git
!mv "/content/stylegan2-ada/stylegan2ada-training-on-a-custom-network/train.py" "/content/stylegan2-ada"
%cd /content/

/content
Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
/content/stylegan2-ada
Branch 'google-colab' set up to track remote branch 'google-colab' from 'origin'.
Switched to a new branch 'google-colab'
Cloning into 'stylegan2ada-training-on-a-custom-network'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
rm: cannot remove '/content/stylegan2-ada/stylegan2ada-training-on-a-custom-network': Is a directory
/content


##Loading zip file to colab

Before using this you must upload the zip file of your dataset to your google drive

The code below will then unzip your dataset

Then you must:

- Enter the full path of your file after the '!unzip '
- Enter after '-d ' what you would like your dataset directory to be named

In [ ]:
!unzip /content/drive/MyDrive/datasets/Dogimages.zip -d dataset

Archive:  /content/drive/MyDrive/datasets/Dogimages.zip
  inflating: dataset/Dogimages/0.jpg  
  inflating: dataset/Dogimages/1.jpg  
  inflating: dataset/Dogimages/10.jpg  
  inflating: dataset/Dogimages/100.jpg  
  inflating: dataset/Dogimages/1000.jpg  
  inflating: dataset/Dogimages/1001.jpg  
  inflating: dataset/Dogimages/1002.jpg  
  inflating: dataset/Dogimages/1003.jpg  
  inflating: dataset/Dogimages/1004.jpg  
  inflating: dataset/Dogimages/1005.jpg  
  inflating: dataset/Dogimages/1006.jpg  
  inflating: dataset/Dogimages/1007.jpg  
  inflating: dataset/Dogimages/1008.jpg  
  inflating: dataset/Dogimages/1009.jpg  
  inflating: dataset/Dogimages/101.jpg  
  inflating: dataset/Dogimages/1010.jpg  
  inflating: dataset/Dogimages/1011.jpg  
  inflating: dataset/Dogimages/1012.jpg  
  inflating: dataset/Dogimages/1013.jpg  
  inflating: dataset/Dogimages/1014.jpg  
  inflating: dataset/Dogimages/1015.jpg  
  inflating: dataset/Dogimages/1016.jpg  
  inflating: dataset/Dogimages

##Preparing your images (once)

This is my own code i wrote for using the Python Imaging Library to add padding to your images

This code will simply run through a directory of jpgs (i.e the dataset you just unzipped) and add padding to the image in order to increase its resolution to a resolution that StyleGAN can work with

To use:
- Set the path variable to the path of your dataset folder
- Ensure there are only images within this folder, if they are jpg that is perfect, if they are png then change which new_im.save line of code you are using
- Then simply run the program
- If any errors are encountered, eg finding a non image file, simply remove the file (using the code in the next cell) then continue

In [ ]:
from PIL import Image
import time
import os, sys

path = "/content/dataset/Dogimages/" #Input the path here of your folder of images, if there are any other files in the folder the program could stop working
dirs = os.listdir( path )
count = 0
desired_size = 512 #Please edit this to be your desired resolution
def resize():
    for item in dirs:
        filename=path+item
        if os.path.isfile(path+item):
            #Check if it is already resized before doing anything
            if (item[len(item)-5]!='r'):
                im = Image.open(filename)
                #Get size of image
                old_size = im.size

                ratio = float(desired_size)/max(old_size)
                new_size = tuple([int(x*ratio) for x in old_size])
                #this part ensures the image is rgb as an error would be thrown if it isnt
                if im.mode != 'RGB':
                    im = im.convert('RGB')
                f, e = os.path.splitext(filename)
                #Resizing 
                im = im.resize(new_size, Image.ANTIALIAS)
                #Making a blank white image
                new_im = Image.new("RGB", (desired_size, desired_size))
                #Putting the original image on the desired size white image
                new_im.paste(im, ((desired_size-new_size[0])//2,
              
                    (desired_size-new_size[1])//2))
                #Save resized file
               #new_im.save(f +'r.png', 'png', quality=90)
                new_im.save(f +'r.jpg', 'jpeg', quality=90)
                #             ^^^^this bit is the file name, im just doing the original name with r after it for resized
                #Delete original once done
                remove_file(filename)
                print (filename+" padded")
    print ("Completed Padding") 
#This function reattempts the delete until the file is available for deleting, stops a random error
def remove_file(path, retries=3, sleep=0.1):
    for i in range(retries):
        try:
            os.remove(path)
        except WindowsError:
            time.sleep(sleep)
        else:
            break

resize()


Completed Padding


If any image files end up being corrupted or a non image file is found and an error is thrown

Simply use !rm and then the path name of the file to remove it

In [ ]:
!rm /content/dataset/PetImages/Cat/666.jpg

rm: cannot remove '/content/dataset/PetImages/Cat/666.jpg': No such file or directory


## Creating dataset from images (once)

Reference: https://github.com/NVlabs/stylegan2-ada#preparing-datasets

### Prepare datasets for StyleGAN2

In the cells below please change:

- custom_dataset to be the file path on the local machine that your tensorflow(tf) records (dataset files that styleGAN2-ada can read) will be stored to 

- custom_images to be the file path of the folder containing your dataset

In [ ]:
custom_dataset = '/content/tfrecords/dog/'
custom_images = '/content/dataset/Dogimages'

This code simply creates the tensorflow records from the images folder, no extra tinkering needed just run the cell

In [ ]:
 !python stylegan2-ada/dataset_tool.py create_from_images {custom_dataset} {custom_images}

Loading images from "/content/dataset/Dogimages"
Creating dataset "/content/tfrecords/dog/"
Added 3000 images.


### Archive prepared datasets to Google Drive

Once you have done this it is a good idea to run this command do save the created tfrecords to your google drive so that they are not lost once you disconnect from the virtual machine

In [ ]:
 colab_transfer.copy_folder('/tfrecords/dog/',
                            source=lm,
                            destination=gd)

Folder /content/drive/My Drive/tfrecords/dog/ already exists. Copy skipped.


## Import previously prepared datasets from Google Drive

This cell is for copying tfrecords which you have previously saved to your drive, back onto the local machine, simply enter the file path

In [ ]:
colab_transfer.copy_folder('tfrecords/dog')

Folder /content/tfrecords/dog already exists. Copy skipped.


## Train new networks

Having produced the dataset in tfrecords form which StyleGAN2-ada can read, this is where we actually train the network.

Note that anything starting with '--' is a training setting

Generally the settings that i have already set should work but read through the next sections to understand them

### Computation time

A tick requires 4 minutes of computation time, so we save a snapshot every 6 ticks (`--snap=6`) in order to save every 24 minutes.


### Documentation for transfer learning

Transfer learning is where we use a pretrained network and train it upon our dataset. It has many advantages but the main one is that it saves on training time

```
transfer learning:
  --resume RESUME       Resume from network pickle (default: noresume)
  --freezed INT         Freeze-D (default: 0 discriminator layers)
```
```
transfer learning source networks (--resume):
  ffhq256        FFHQ trained at 256x256 resolution. (256x256 faces)
  ffhq512        FFHQ trained at 512x512 resolution. (512x512 faces)
  ffhq1024       FFHQ trained at 1024x1024 resolution. (1024x1024 faces)
  celebahq256    CelebA-HQ trained at 256x256 resolution. (256x256 celeb faces)
  lsundog256     LSUN Dog trained at 256x256 resolution. (256x256 diverse dataset - recommended)
  <path or URL>  Custom network pickle.
Note that you will have to use a source which is of the same resolution
```

### Training recommendations

-   about `--gamma`:
> The training configuration can be further customized with additional arguments. Common examples:
> * `--gamma=10` overrides R1 gamma. We strongly recommend trying out at least a few different values for each new dataset.
-   about `--cfg`:
> Please note that `--cfg=auto` only serves as a reasonable first guess for the hyperparameters &mdash; it does not necessarily lead to optimal results for a given dataset. 
-   about `--metrics`:
> By default, `train.py` will automatically compute FID for each network pickle. We strongly recommend inspecting `metric-fid50k_full.txt` at regular intervals to monitor the training progress. When desired, the automatic computation can be disabled with `--metrics none` to speed up the training.

For your initial run just leave all of these as they are.



### Training settings

Resolution is automatically determined from the dataset.
Other settings are chosen as follows:
- horizontal mirroring, via `--mirror=true` (instead of default to False),
- no EMA rampup, via `--cfg=auto_no_ramp`,
- cfg map = 8 (instead of 2), via `--cfg_map=8`,
- freezed D (the first 10 layers, as in the article for resolution 256), via `--freezed=10`.

References:
-   https://github.com/NVlabs/stylegan2-ada#training-new-networks
-   https://raw.githubusercontent.com/NVlabs/stylegan2-ada/main/docs/train-help.txt

### Train

NB: to ensure your Colab session stays connected:

- Start running the cell you want to run
- Left click on the cell which says 'while True:pass'
- At the top click runtime, then select run after
- This means that once the run has completed this cell will automatically run and colab from disconnecting
- Note that the 8 hour limitation still exists

#### Transfer Learning Run

- Set your outdir variable to where you would like your trained network to be saved, it is strongly recommended to save it to your google drive to ensure it is not lost due to disconnecting from the local machine

- set the data variable to your tfrecords file

- Choose your --resume parameter based on which dataset under 'documentation for transfer learning' matches your resolution

- 

In [ ]:
 !python stylegan2-ada/train.py \
  --outdir='/content/drive/My Drive/training-runs' \
  --snap=6 \
  --data='/content/tfrecords/dog' \
  --mirror=true \
  --metrics=none \
  --cfg=auto \
  --kimg=5000 \
  --resume=ffhq512 \
  --freezed=10 \
 

tcmalloc: large alloc 4294967296 bytes == 0x8a60000 @  0x7f33cab67001 0x7f33c7daa4ff 0x7f33c7dfab08 0x7f33c7dfeac7 0x7f33c7e9d1a3 0x50a4a5 0x50cc96 0x507be4 0x508ec2 0x594a01 0x549e8f 0x5515c1 0x5a9dac 0x50a433 0x50cc96 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x5095c8 0x50a2fd 0x50beb4 0x507be4
tcmalloc: large alloc 4294967296 bytes == 0x7f31dc2c2000 @  0x7f33cab651e7 0x7f33c7daa41e 0x7f33c7dfac2b 0x7f33c7dfb30f 0x7f33c7e9d0a3 0x50a4a5 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4
tcmalloc: large alloc 4294967296 bytes == 0x7f30db2c0000 @  0x7f33cab651e7 0x7f33c7daa41e 0x7f33c7dfac2b 0x7f33c7dfb30f 0x7f33736ea0c5 0x7f337306d902 0x7f337306deb2 0x7f3373026c3e 0x50a12f 0x50beb4 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x588c8b 0x59fd0e 0x50d256 0x507b

In [ ]:
while True:pass

#### Following runs (resume from latest snapshot)

If you are still connected to the machine you will **automatically** resume from the latest pickle snapshot.

If not you will have to manually specfiy the transfer learning folder path after --resume

Note that you will have to **manually** specify the augmentation strength, as found in `log.txt`.

In [ ]:
!python stylegan2-ada/train.py \
 --outdir='/content/drive/My Drive/training-runs' \
 --snap=6 \
 --data='/content/tfrecords/dog' \
 --mirror=true \
 --metrics=none \
 --cfg=auto \
 --kimg=5000 \
 --resume=latest \
 --aug=fixed \
 --p=0.713 \
 --freezed=10 \
 

2021-02-25 14:23:26.065255: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "stylegan2-ada/train.py", line 18, in <module>
    import dnnlib.tflib as tflib
  File "/content/stylegan2-ada/dnnlib/tflib/__init__.py", line 9, in <module>
    from . import autosummary
  File "/content/stylegan2-ada/dnnlib/tflib/autosummary.py", line 32, in <module>
    from . import tfutil
  File "/content/stylegan2-ada/dnnlib/tflib/tfutil.py", line 18, in <module>
    import tensorflow.contrib   # requires TensorFlow 1.x!
ModuleNotFoundError: No module named 'tensorflow.contrib'


In [ ]:
while True:pass